In [37]:
import os, requests
from dotenv import load_dotenv
import pandas as pd
import time
import json

In [40]:
load_dotenv()
API_KEY = os.getenv("API_KEY")
DISCOVER_URL = "https://api.themoviedb.org/3/discover/movie"

movie_id = []
for page in range(1, 3):
    resp = requests.get(DISCOVER_URL, params={
        "api_key": API_KEY,
        "with_original_language": "en",
        "primary_release_date.gte": "1970-01-01",
        "sort_by": "popularity.desc",
        "page": page
    }).json()

    results = resp.get("results", [])
    if not results:
        print('Error: Could not find results')
        break

    movie_id.extend([m["id"] for m in resp.get("results", [])])
    print(f"Page {page}/1000 → total {len(movie_id)} movies")
    time.sleep(0.25)

df_ids = pd.DataFrame({"id": movie_id})
df_ids.to_csv('movie_id.csv', index=False)

Page 1/1000 → total 20 movies
Page 2/1000 → total 40 movies


In [33]:
BASE_DETAIL = "https://api.themoviedb.org/3/movie/{}"

records = []

for idx, mid in enumerate(df_ids["id"], start=1):
    resp = requests.get(BASE_DETAIL.format(mid), params={
        "api_key": API_KEY,
        "append_to_response": "credits.cast,keywords,similar,recommendations,release_dates"
    }).json()
    records.append(resp)
    if idx % 100 == 0:
        print(f"Fetched {idx}/{len(df_ids)}")
    time.sleep(0.25)

print(records)

[{'adult': False, 'backdrop_path': '/gsQJOfeW45KLiQeEIsom94QPQwb.jpg', 'belongs_to_collection': None, 'budget': 0, 'genres': [{'id': 28, 'name': 'Action'}, {'id': 53, 'name': 'Thriller'}], 'homepage': '', 'id': 1125899, 'imdb_id': 'tt27812086', 'origin_country': ['GB'], 'original_language': 'en', 'original_title': 'Cleaner', 'overview': "When a group of radical activists take over an energy company's annual gala, seizing 300 hostages, an ex-soldier turned window cleaner suspended 50 storeys up on the outside of the building must save those trapped inside, including her younger brother.", 'popularity': 451.8095, 'poster_path': '/mwzDApMZAGeYCEVjhegKvCzDX0W.jpg', 'production_companies': [{'id': 138743, 'logo_path': '/abFJGmtPFs6Bq5cBavmrnXDcpf8.png', 'name': 'Anton', 'origin_country': 'GB'}, {'id': 892, 'logo_path': '/s7eFTMS1udDI1EG0Zxlx4ylx1Nl.png', 'name': 'Qwerty Films', 'origin_country': 'GB'}], 'production_countries': [{'iso_3166_1': 'GB', 'name': 'United Kingdom'}], 'release_date'

In [39]:
BASE_DETAIL = "https://api.themoviedb.org/3/movie/{}"

with open("movies_raw.ndjson", "a", encoding="utf-8") as fout:
    for idx, mid in enumerate(df_ids["id"], start=1):
        resp = requests.get(BASE_DETAIL.format(mid), params={
        "api_key": API_KEY,
        "append_to_response": "credits.cast,keywords,similar,recommendations,release_dates"
    }).json()
        fout.write(json.dumps(resp) + "\n")  # write one JSON object per line

        if idx % 100 == 0:
            print(f"Fetched {idx}/{len(df_ids)}")
        time.sleep(0.25)

print("Done — saved NDJSON for all movies.")

Done — saved NDJSON for all movies.
